In [29]:
from distutils.command.bdist import show_formats
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from quantuminspire.qiskit import QI
import numpy as np
import utils as u

In [30]:
u.set_auth()

In [31]:
basis_to_bit = {"Z" : "1", "X" : "0"}

In [34]:
def build_circuit(alice_string, bob_bases, qi_backend, n_shots=1024):
    if len(alice_string) != len(bob_bases):
        raise IndexError(
            "Length of bit string and length of bases to measure in do not match."
        )
    else:
        length = len(alice_string)

    circuits = []
    final_bit_string = ""

    for index in range(length):
        qr = QuantumRegister(1)
        cr = ClassicalRegister(1)
        circuit = QuantumCircuit(qr, cr)
        bit = alice_string[index]
        basis = bob_bases[index]

        # Step 1: Initialize qubits according to Alice's bit string
        if bit == "0":  # if the bit is 0
            circuit.i(qr[0])  # we initialize the qubit in the |0> state
        elif bit == "1":  # if the bit is 1
            circuit.h(qr[0])  # we initialize the qubit in the |+> state

        # Step 2: Measure qubits in Bob's chosen bases
        if basis.upper() == "Z":  # if Bob picks the Z basis,
            circuit.i(qr[0])  # we stay in the Z basis
        elif basis.upper() == "X":  # if Bob picks the X basis,
            circuit.h(qr[0])  # we apply a Hadamard gate so that the measurement will be in the Z basis
        circuit.measure(qr, cr)

        # Step 3: run the circuits on the Quantum Inspire backend and compile the results
        qi_job = execute(circuit, backend=qi_backend, shots=n_shots)
        qi_result = qi_job.result()
        histogram = qi_result.get_counts(circuit)

        # print(histogram)

        # num_0 counts eigvl = 1
        # num_1 counts eigvl = -1
        num_0, num_1 = 0, 0 

        if "0" in histogram.keys():
            num_0 = histogram["0"]
        if "1" in histogram.keys():
            num_1 = histogram["1"]

        # print(f"num_0:\t{num_0}\nnum_1:\t{num_1}\n")

        if num_0 >= num_1:
            # we take majority rule, state collapses to +1 eigenvalue of the measurement basis
            # or equal amount of 0 and 1 measurement results, result is indeterminate
            bit = "n"  # bit is indeterminate
        else:
            # we take majority rule, state collapses to -1 eigenvalue of the measurement basis
            bit = basis_to_bit[basis.upper()]

        final_bit_string += bit
        circuits.append(circuit)

    return final_bit_string, circuits


In [35]:
# QI.get_backend("Starmon-5")
final_bit_string, circuits = build_circuit("1001", ["Z", "X", "X", "X"], QI.get_backend("QX single-node simulator"))
final_bit_string

'10nn'